In [1]:
!pip install pydub

In [2]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as idsp
import pydub
import os

In [5]:
train_folder = "/content/sample_data/train"
split_train_folder = "/content/sample_data/split_train"
test_folder = "/content/sample_data/test"
split_test_folder = "/content/sample_data/split_test"
mfccs_folder = "/content/sample_data/mfccs"

os.mkdir(train_folder)
os.mkdir(split_train_folder)
os.mkdir(test_folder)
os.mkdir(split_test_folder)
os.mkdir(mfccs_folder)

## Split audio files

In [23]:
class SoundSegment:
    def __init__(self, start: float, end: float, label: str):
        self.start = start
        self.end = end
        self.label = label

    def __str__(self):
        return "({} - {}: {})".format(self.start, self.end, self.label)

    def __repr__(self):
        return self.__str__()

In [24]:
def split_to_folder(in_folder, out_folder):
    files = os.listdir(in_folder)
    file_names = [file[:-4] for file in files]
    for file in file_names:
        # read labels for that file
        labels = []
        with open(f"{in_folder}/{file}.txt", "r") as f:
            for line in f:
                if line.strip() == "":
                    continue
                start, stop, label = line.split("\t")
                labels.append(SoundSegment(float(start), float(stop), label.strip()))

        # chop each label into its own wav
        for order, label in enumerate(labels):
            audio = pydub.AudioSegment.from_wav(f"{in_folder}/{file}.wav")
            audio = audio[int(label.start * 1000) : int(label.end * 1000)]
            audio.export(
                f"{out_folder}/{label.label}_{file}_{order+1}.wav", format="wav"
            )

In [25]:
split_to_folder(train_folder, split_train_folder)
split_to_folder(test_folder, split_test_folder)

## MFCC 

In [46]:
sr = 22050

def mfccs_extract(file):
    sound, sr = librosa.load(f"{split_train_folder}/{file}.wav")
    mfcc = librosa.feature.mfcc(y=sound, n_mfcc=13, sr=sr)
    for i in range(mfcc.shape[1]):
        mfcc[:,i] = mfcc[:,i] - np.min(mfcc[:,i])
        mfcc[:,i] = mfcc[:,i]/(np.max(mfcc[:,i]) - np.min(mfcc[:,i]))
    delta_mfcc = librosa.feature.delta(mfcc)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    mfccs = np.concatenate((mfcc, delta_mfcc, delta2_mfcc))
    return mfccs

In [47]:
files = os.listdir(split_train_folder)
file_names = [file[:-4] for file in files]
# print(file_names)

for file in file_names:
    mfccs = mfccs_extract(file)
    np.savetxt(f"{mfccs_folder}/{file}.txt", mfccs, delimiter=",")

In [48]:
def load_mfccs(file):
    return np.loadtxt(f"{mfccs_folder}/{file}.txt", delimiter=",")

In [49]:
test_files = list(filter(lambda k: 'test' in k, file_names))
file_names = list(filter(lambda k: 'test' not in k, file_names))

A_files = list(filter(lambda k: 'A' in k, file_names))
B_files = list(filter(lambda k: 'B' in k, file_names))
len_files = list(filter(lambda k: 'len' in k, file_names))
xuong_files = list(filter(lambda k: 'xuong' in k, file_names))
trai_files = list(filter(lambda k: 'trai' in k, file_names))
phai_files = list(filter(lambda k: 'phai' in k, file_names))
ban_files = list(filter(lambda k: 'ban' in k, file_names))
nhay_files = list(filter(lambda k: 'nhay' in k, file_names))
sil_files = list(filter(lambda k: 'sil' in k, file_names))

In [50]:
def load_mfccs(label_files):
    mfccs = []
    for file in label_files:
        mfccs.append(load_mfccs(file))
    return mfccs

In [51]:
A_mfccs = load_mfccs(A_files)
B_mfccs = load_mfccs(B_files)
len_mfccs = load_mfccs(len_files)
xuong_mfccs = load_mfccs(xuong_files)
trai_mfccs = load_mfccs(trai_files)
phai_mfccs = load_mfccs(phai_files)
ban_mfccs = load_mfccs(ban_files)
nhay_mfccs = load_mfccs(nhay_files)
sil_mfccs = load_mfccs(sil_files)

test_mfccs = load_mfccs(test_files)

## DTW

In [52]:
def dtw_cost(mfcc1, mfcc2):
    D, wp = librosa.sequence.dtw(mfcc1, mfcc2, subseq = True, metric='euclidean')
    return D[-1, -1]/wp.shape[0]

In [53]:
def align_mfccs(label_mfccs):
    template_mfcc = label_mfccs[0]
    other_templates = label_mfccs[1:]
    frame_num = template_mfcc.shape[1]
    frame_lst = []
    for i in range(frame_num):
        frame = [template_mfcc[:, i]]
        frame_lst.append(frame)

    for mfcc in other_templates:
        D, wp = librosa.sequence.dtw(template_mfcc, mfcc, subseq = True, metric='euclidean')
        for i in wp:
            frame_lst[i[0]].append(mfcc[:,i[1]])
    return frame_lst

In [54]:
def average_templates(label_mfccs):
    frame_lst = align_mfccs(label_mfccs)
    frame_num = len(frame_lst)
    average_mfcc = np.zeros(shape=(frame_num, 39))
    for i in range(frame_num):
        frame_average = np.array(frame_lst[i]).mean(axis=0)
        average_mfcc[i] = frame_average
    return average_mfcc.transpose()

In [55]:
A_template = average_templates(A_mfccs)
B_template = average_templates(B_mfccs)
len_template = average_templates(len_mfccs)
xuong_template = average_templates(xuong_mfccs)
trai_template = average_templates(trai_mfccs)
phai_template = average_templates(phai_mfccs)
nhay_template = average_templates(nhay_mfccs)
ban_template = average_templates(ban_mfccs)
sil_template = average_templates(sil_mfccs)

In [ ]:
dict = {
  0: "A",
  1: "B",
  2: "len",
  3: "xuong",
  4: "trai",
  5: "phai",
  6: "ban",
  7: "nhay",
  8: "sil"
}

In [131]:
test_num = 0
sound_test = test_mfccs[test_num]

def costs_compute(sound):
    A_cost = dtw_cost(A_template, sound)
    B_cost = dtw_cost(B_template, sound)
    len_cost = dtw_cost(len_template, sound)
    xuong_cost = dtw_cost(xuong_template, sound)
    trai_cost = dtw_cost(trai_template, sound)
    phai_cost = dtw_cost(phai_template, sound)
    ban_cost = dtw_cost(ban_template, sound)
    nhay_cost = dtw_cost(nhay_template, sound)
    sil_cost = dtw_cost(sil_template, sound)
    return [A_cost, B_cost, len_cost, xuong_cost, trai_cost, phai_cost, ban_cost, nhay_cost, sil_cost]

costs = costs_compute(sound_test)
result = dict.get(np.array(costs).argmin())
print('result = \"{}\"'.format(result))
print('filename = \"{}\"'.format(test_files[test_num].split('_')[0]))
print("Correct =  {}".format(result == test_files[test_num].split('_')[0]))

result = "trai"
filename = "trai"
Correct =  True


In [129]:
test_len = len(test_mfccs)
correct = 0

for i in range(test_len):
    sound = test_mfccs[i]
    costs = costs_compute(sound)
    result = dict.get(np.array(costs).argmin())
    label = test_files[i].split('_')[0]
    if(result == label):
        correct += 1

print("Accuracy: {}/{}".format(correct, test_len))

Accuracy: 120/121


## HMM

In [10]:
!pip install hmmlearn

     |████████████████████████████████| 129 kB 5.1 MB/s 


In [11]:
from hmmlearn import hmm
import librosa
import numpy as np
import pandas as pd
from IPython import display
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [47]:
def get_mfcc(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)
    return np.vstack((mfcc, delta, delta2)).T

In [48]:
def buildDataSet(folder):
    dataset = {}
    files = os.listdir(folder)
    file_names = [file[:-4] for file in files]
    for file in file_names:
        y, sr = librosa.load(f"{folder}/{file}.wav")
        mfccs = get_mfcc(y, sr)
        label = file.split('_')[0]
        if label not in dataset.keys():
            dataset[label] = []
            dataset[label].append(mfccs)
        else:
            exist_feature = dataset[label]
            exist_feature.append(mfccs)
            dataset[label] = exist_feature
    return dataset

In [49]:
def train_GMMHMM(dataset):
    GMMHMM_Models = {}
    states_num = 5
    GMM_mix_num = 3
    tmp_p = 1.0/(states_num-2)
    transmatPrior = np.array([[tmp_p, tmp_p, tmp_p, 0 ,0], \
                               [0, tmp_p, tmp_p, tmp_p , 0], \
                               [0, 0, tmp_p, tmp_p, tmp_p], \
                               [0, 0, 0, 0.5, 0.5], \
                               [0, 0, 0, 0, 1]],dtype=np.float)

    startprobPrior = np.array([0.5, 0.5, 0, 0, 0],dtype=np.float)

    for label in dataset.keys():
        model = hmm.GMMHMM(n_components=states_num, n_mix=GMM_mix_num, \
                           transmat_prior=transmatPrior, startprob_prior=startprobPrior, \
                           covariance_type='diag', n_iter=10)
        trainData = dataset[label]
        length = np.zeros([len(trainData), ], dtype=np.int)
        for m in range(len(trainData)):
            length[m] = trainData[m].shape[0]
        trainData = np.vstack(trainData)
        model.fit(trainData, lengths=length)  # get optimal parameters
        GMMHMM_Models[label] = model
    return GMMHMM_Models

In [ ]:
train_dataset = buildDataSet(split_train_folder)
hmmModels = train_GMMHMM(train_dataset)

In [51]:
testDataSet = buildDataSet(split_test_folder)

In [94]:
dict = {}
for i, k in enumerate(testDataSet.keys()):
    dict[i] = k

dict

{0: 'trai',
 1: 'sil',
 2: 'xuong',
 3: 'len',
 4: 'A',
 5: 'phai',
 6: 'B',
 7: 'ban',
 8: 'nhay'}

In [103]:
from sklearn.metrics import accuracy_score

y_true = []
y_preds = []

for label in testDataSet.keys():
    for mfcc in testDataSet[label]:
        scores = [hmmModels[label].score(mfcc) for label in testDataSet.keys()]
        preds = np.argmax(scores)
        y_true.append(label)
        y_preds.append(dict.get(preds))

accuracy = accuracy_score(y_true, y_preds)
print("Accuracy Score: {}".format(accuracy))

Accuracy Score: 1.0
